In [48]:
class Node:
  def __init__(self, name):
    self.name:str  = name
    self.prev:Node = None
    self.next:Node = None

  def __str__(self):
    outp = f'Node: {self.name}\n\tPrev: {self.prev.name}\n\tNext: {self.next.name}'

    return outp

class Network:
  def __init__(self):
    self._nodes = {}
    self._services = {
                      # service_name: (node_name, [security_level, wire_cost, wifi_cost])
                      'Autenticacao dos usuarios':('A', [2, 0, 4]),
                      'Criptografia dos dados':('C', [2, 0, 1]),
                      'Monitoramento da velocidade da rede':('E', [0, 0, 5]),
                      'Compartilhamento de arquivos':('G', [2, 0, 1])
                     }

  def add_node(self, node_name: str) -> None:
    node = Node(node_name)

    self._nodes[node_name] = node

  def add_connection(self, node_name: str, prev: Node, next: Node) -> None:
    self._nodes[node_name].prev = self._nodes[prev]
    self._nodes[node_name].next = self._nodes[next]

  def print_network(self) -> None:
    for i in self._nodes.keys():
      print(f'Node: {i}\n\tPrev: {self._nodes[i].prev.name}\n\tNext: {self._nodes[i].next.name}')

  def bfs(self, start_node: str, target_node: str) -> list|None:
    if not start_node:
      return None

    visited = set()
    queue = []
    queue.append((self._nodes[start_node], []))  # [(node, path), ...]

    while queue:
      current, path = queue.pop(0)

      if current.name == target_node:
        return path + [current.name]

      if current.name not in visited:
        visited.add(current)

        if current.next:
          queue.append((current.next, path + [current.name]))
        if current.prev:
          queue.append((current.prev, path + [current.name]))

    return None

  # Selects the best traffic path through the network, ensuring efficiency and security
  def best_path(self, service: str, target_info: list, source: str) -> str:
    path = ''

    path_bfs = self.bfs(source, target_info[0])
    target_info[1][1] = len(path_bfs) - 1 # Calculate cost to send by wire

    # Request
    # Wifi is cheaper (and don't need high security)
    if target_info[1][2] < target_info[1][1] and target_info[1][0] < 2:
      path = f'{source} -> Wifi -> {target_info[0]}'
    else:
      path = ' -> '.join(path_bfs)

    # Response
    # 'Criptografia de dados' always need send by wire to maintain security
    if target_info[1][2] < target_info[1][1] and service != 'Criptografia dos dados':
      path += f'\n{target_info[0]} -> Wifi -> {source}'
    else:
      if path_bfs:
         path_bfs.reverse()
         sec_path = path_bfs
      else:
        sec_path = self.bfs(target_info[0], source)

      path += '\n' + ' -> '.join(sec_path)

    return path

  # Seeks the optimal path from a source node and one (or two) service(s)
  def find_path(self, source: str, service1: str, service2: str=None) -> str:
    target1 = self._services.get(service1)
    path = ''

    # Service/Node not found
    if not target1:
      return 'Service not found'
    if source not in self._nodes.keys():
      return 'Node not found'

    # 1 service
    if service2 == None:
      # Secure connection
      if target1[1][0] == 2:
        target_auth = self._services.get('Autenticacao dos usuarios')
        path = self.best_path('Autenticacao dos usuarios', target_auth, source)

        if service1 != 'Autenticacao dos usuarios':
          path += '\n' + self.best_path(service1, target1, source)
      # Unsecure connection
      else:
        path = self.best_path(service1, target1, source)
    # 2 services
    else:
      target2 = self._services.get(service2)

      if target1[1][0] == 2 or target2[1][0] == 2:
        target_auth = self._services.get('Autenticacao dos usuarios')
        auth = self.best_path('Autenticacao dos usuarios', target_auth, source)

      if target1[1][0] == 2:
        path += auth + '\n'
        if service1 != 'Autenticacao dos usuarios':
           path += self.best_path(service1, target1, source) + '\n'
      else:
        path += self.best_path(service1, target1, source) + '\n'

      if target2[1][0] == 2:
        if target1[1][0] < 2:
          path += auth + '\n'
        if service2 != 'Autenticacao dos usuarios':
           path += self.best_path(service2, target2, source)
      else:
        path += self.best_path(service2, target2, source)

    return path

In [49]:
# Network creation
ring = Network()

ring.add_node('A')
ring.add_node('B')
ring.add_node('C')
ring.add_node('D')
ring.add_node('E')
ring.add_node('F')
ring.add_node('G')
ring.add_node('H')

ring.add_connection('A', 'H', 'B')
ring.add_connection('B', 'A', 'C')
ring.add_connection('C', 'B', 'D')
ring.add_connection('D', 'C', 'E')
ring.add_connection('E', 'D', 'F')
ring.add_connection('F', 'E', 'G')
ring.add_connection('G', 'F', 'H')
ring.add_connection('H', 'G', 'A')

In [53]:
source = input().split(':')[1].strip()
service = input().split(':')[1].strip()

path = ring.find_path(source, service)
if path:
  print(path)
else:
  print('Service not found')

Nó: G
Serviço: Criptografia dos dados
G -> H -> A
A -> H -> G
G -> H -> A -> B -> C
C -> B -> A -> H -> G


In [52]:
source = input().split(':')[1].strip()
service1 = input().split(':')[1].strip()
service2 = input().split(':')[1].strip()

path = ring.find_path(source, service1, service2)
if path:
  print(path)
else:
  print('Service not found')

Nó: F
Serviço 1: Monitoramento da velocidade da rede
Serviço 2: Compartilhamento de arquivos
F -> E
E -> F
F -> G -> H -> A
A -> H -> G -> F
F -> G
G -> F
